# Assignment 3

Στόχος μας είναι να προβλέψουμε την τιμή (Χ,6) του User X για το Movie 6.

<b>Cosine similarity imported from scikitlearn</b>
[Click here for more info](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)  


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Setting 'User' col as an index 
data = pd.read_csv("ask3.csv", index_col='User')
df = pd.DataFrame(data)
df

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6
User,,,,,,
X,5,3.0,NaN,1.0,NaN,NaN
Y,4,2.0,1.0,NaN,NaN,1.0
Z,3,NaN,NaN,1.0,3.0,3.0
W,2,5.0,1.0,5.0,3.0,4.0


## Cosine Similarity between Users
In order to use cosine similarity we have to replace NaN values with 0's

We can see that User X is pretty much identical to User Y (0.93697496 ) and then User W (0.56694671)  
<b>Topk(u) = {Y:0.93697496, W:0.51110125}</b>

In [3]:
print(cosine_similarity(df.fillna(0),df.fillna(0)))

[[1.         0.93697496 0.51110125 0.56694671]
 [0.93697496 1.         0.60436722 0.54824099]
 [0.51110125 0.60436722 1.         0.6761234 ]
 [0.56694671 0.54824099 0.6761234  1.        ]]


<b>Estimating User X ratings based on ratings of 𝑇𝑜𝑝𝐾(u) using some aggregation function.</b>
$$\widehat{r_{u,i}} =\frac{1}{z} \sum_{v\epsilon Topk(u)}sim(u,v)r_{v,i}$$
$$z = \sum_{v\epsilon Topk(u)}sim(u,v)$$

<i>Answer:</i>$$\widehat{r_{X,Movie6}} = 2.130936646454466$$

In [4]:
simx_y = 0.93697496
simx_w = 0.56694671

z = simx_y + simx_w
sumd = (df.loc['Y'][-1] * simx_y) + (df.loc['W'][-1] * simx_w)

rating_x6 = sumd/z

print("Estimated rating for User X-Movie 6: "+ str(rating_x6))

Estimated rating for User X-Movie 6: 2.130936646454466


## Normalized cosine similarity  
is done by subtracting the mean before computing the cosine formula.

In [5]:
#Calculating mean for every user and subtracting (via lambda function)
def f():
    modf = df.copy()
    for i,r in df.iterrows():
        mean = r.sum()/r.count()
        modf.loc[i] = r.apply(lambda x: x-mean)
    return modf

df1 = f()

df1

,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6
User,,,,,,
X,2.000000,0.000000,NaN,-2.000000,NaN,NaN
Y,2.000000,0.000000,-1.000000,NaN,NaN,-1.000000
Z,0.500000,NaN,NaN,-1.500000,0.500000,0.500000
W,-1.333333,1.666667,-2.333333,1.666667,-0.333333,0.666667


After subtracting the mean rating per user and then compute Cosine,  
TopK(Users) are now Z and Y.  
<b>topK(U) = {Z:0.81649658, Y:0.57735027}</b>

In [6]:
print(cosine_similarity(df1.fillna(0),df1.fillna(0)))

[[ 1.          0.57735027  0.81649658 -0.5809475 ]
 [ 0.57735027  1.          0.11785113 -0.1118034 ]
 [ 0.81649658  0.11785113  1.         -0.47434165]
 [-0.5809475  -0.1118034  -0.47434165  1.        ]]


<b>Deviations:</b>  
$$\widehat{r_{u,i}} =\overline{r_{u}} + \frac{1}{z} \sum_{v\epsilon Topk(u)}sim(u,v)(r_{v,i} - \overline{r_{v}})$$  

<i>Answer:</i>  
$$\widehat{r_{X,Movie6}} = 2.8786796555159557$$

In [7]:
simx_y = 0.57735027
simx_z = 0.81649658

z = simx_y + simx_z
mean_rating_x = df.loc['X'].sum()/df.loc['X'].count()
mean_rating_y = df.loc['Y'].sum()/df.loc['Y'].count()
mean_rating_z = df.loc['Z'].sum()/df.loc['Z'].count()

y_rating_m6 = df.loc['Y'][-1]
z_rating_m6 = df.loc['Z'][-1]

dev = simx_y * (y_rating_m6 - mean_rating_y) + (simx_z* (z_rating_m6 - mean_rating_z))

dev = dev/z
print("Deviation = " + str(dev))
r = mean_rating_x + dev
print("Estimated rating for UserX-Movie6 with deviations: "+ str(r))

Deviation = -0.1213203444840443
Estimated rating for UserX-Movie6 with deviations: 2.8786796555159557


## Συμπέρασμα

Πέρα απο τη διαφορά στους πιο παρόμοιους χρήστες ({Y,W} στο 1ο , {Y,Z} στο 2ο) παρατηρούμε και μια αύξηση στην εκτίμηση της βαθμολογίας του User X (Movie6).  
Ο λογος ειναι επειδή στο 2ο παίρνουμε υπόψιν μας το πόσο απέχουν συνολικα οι βαθμολογιες των topK(U) για το Movie6 (ως προς το πληθος topk) και το πρoσθετουμε στο μεσο ορο βαθμολόγησης του Χ.  
Με άλλα λόγια υπάρχει μια εκτίμηση στο πως βαθμολόγει ο κάθε χρήστης (π.χ Μπορει σε 2 άτομα να άρεσε η ίδια ταινία αλλα ο ένας να βαθμολογεί γενικα πιο χαμήλα τις ταινίες)